ft-make data work


input

<user>what is the capital of France?</user>
<assistant>Paris</assistant>
<user>what about Spain?</user>
<assistant>Madid</assistant>
<user>Germany?</user>

target output
berlin


Input Alice has 3 apples and buys 2 more. How many now?

Target Output

<think> Start with 3. Buys 2 ⇒ 3+2=5. </think>

<answer>5</answer>


Input 
Carly has 8 apples and buys 2 more, but then sells 5 to the local baker. How many now?

Model Output 
8+2-5=5 
Answer: 5

Grader 
Correct: +1 
Shows work: +1 
Total reward (score): +2

Input
Greet politely

Model Output
Hi there! How are you?

Grader
High score on politeness: +1
High score on enthusiasm: +1
High score on engagement: +1
Total reward (score): +3


In [ ]:
dataset = load_dataset('openai','main')
train = dataset['train']
test = dataset['test']

In [ ]:
input_ids = tokenizer(text)

output_text = tokenizer.decode(output_text)

In [ ]:
#from probabilities to next token id

model.generate(**ids) #greedy selection

model.generate(**ids,do_sample=true,temperature) #sample
# temperature = 0 时 就相当于greedy selection

#beam search

model.generate(**ids , num_beams=2)


In [ ]:
AutoTokenizer.from_pretrained(module_name)(prompt_batch , padding=True)['input_ids']


tensor([[0,  0,  2640,   3073,    418,    3221,  21142],
        [0,  0,  3073,    418,   3221,    21142,    30],
        [0,  0,     0,      0,   2640,    3073,    418]])

In [ ]:
#加掩码算损失

import torch
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss(ignore_index = -100) #遇到数字-100 直接跳过

logits = torch.randn(1,6,1000) # batch_size seq_len vocab_size

labels = torch.tensor([[-100, -100, -100,  -100,  45,  982]])

loss = loss_fn(logits.view(-1,1000),  labels.view(-1))
#.view()  重塑张量形状 
# view(-1请pytorch自行计算剩下的维度合并起来时多少,词表大小)处理输出logits
#view(-1 把所有元素拉成一排)处理标签
loss.backward()


In [ ]:
training_args = SFTConfig(completion_only_loss = True , ...)

trainer = SFTTrainer(
    model = model_name,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    learning_rate = 2e-6,
    weight_decay = 0.01,
    per_device_train_batch_size = 20,
    per_device_eval_batch_size = 20,
    num_train_epochs = 3,
)

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)           # 为当前 GPU 设置种子
    torch.cuda.manual_seed_all(seed)       # 为所有 GPU 设置种子（多卡训练）
    
    # 强制 CuDNN 使用确定性算法（会让训练变慢，但保证结果完全一致）
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
    

固定随机种子

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraModel , LoraConfig

config = LoraConfig(
    task_type = "SEQ_2_SEQ_LM",
    r = 8,
    lora_alpha = 32,
    target_modules = ["q" , "v"]
)

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
lora_model = LoraModel(model,config,"default")
